In [1]:
import torch
from torch import nn
from torch.utils.data import dataset as Dataset
from torch.utils.data import dataloader as DataLoader
import pandas as pd
import numpy as np
import os

from matplotlib import pyplot as plt
import seaborn as sns

In [2]:
#testcell for conflict git branches
torch.tensor([[2, 3, 2], [1, 6, 5]])

tensor([[2, 3, 2],
        [1, 6, 5]])

In [3]:
# test = %pwd
data = pd.read_csv('D:/Code_software/Jupyter_Notebook/kaggle1_data/train.csv')
# print(data.duplicated().sum()) #=0
newdf = data.interpolate()
diff = newdf['Age'].compare(data['Age'])
diff.index

Index([  5,  17,  19,  26,  28,  29,  31,  32,  36,  42,
       ...
       832, 837, 839, 846, 849, 859, 863, 868, 878, 888],
      dtype='int64', length=177)

In [4]:
###计算熵增益 H(Survived|Pclass) = -p1log2p1-p2log2p2
#####################耻辱柱：条件不要加df[]，会变成df&df报错！！！
# survive_cls1 = newdf[(newdf['Pclass']==1) & (newdf['Survived']==1)]
# cls1 = newdf[newdf['Pclass']==1]

# lenth = len(newdf)
# pct = [[] for i in range(2)] #0.63, 0.47, 0.24   H(S|1)  H(S|2)  H(S|3)
# for i in range(1, 4):
#     all_mem = len(newdf[newdf['Pclass']==i])
#     survive = len(newdf[(newdf['Pclass']==i) & (newdf['Survived']==1)])
#     sur_pct = round(survive/all_mem, 2) #活下来的占该cls的比例
#     mem_pct = round(all_mem/lenth, 2) #该class占总的比例
#     pct[0].append(sur_pct)
#     pct[1].append(mem_pct)

# HSx_1 = round(-np.log2(pct[0][0]) * pct[0][0] - np.log2(1 - pct[0][0]) * (1 - pct[0][0]), 2) #0.95
# HSx_2 = round(-np.log2(pct[0][1]) * pct[0][1] - np.log2(1 - pct[0][1]) * (1 - pct[0][1]), 2) #1
# HSx_3 = round(-np.log2(pct[0][2]) * pct[0][2] - np.log2(1 - pct[0][2]) * (1 - pct[0][2]), 2) #0.8
# HSx = pct[1][0] * HSx_1 + pct[1][1] * HSx_2 + pct[1][2] * HSx_3 #0.878
# HS = round(-np.log2(0.3)*0.3 * 3, 2) #1.56
# HS_gain = HS - HSx #熵增益
# HS_gain

In [5]:
newdf.loc[newdf['Sex']=='male', 'Sex'] = 0
newdf.loc[newdf['Sex']=='female', 'Sex'] = 1

In [6]:
#只能用在dummies前的dataset，和上面的cell同时用才能看性别

# def check_HS_gain(col, df):
#     assert type(col) == str
#     typ = list(df.groupby(col).groups.keys())
#     num = range(len(typ))
#     HS_x = []
#     sur_pc = []
#     for j in num:
#         ind = typ[j]
#         condition1 = df['Survived'] == 1
#         condition2 = df[col] == ind
#         intersection = df[(condition1) & (condition2)]
#         sur_pct = round(len(intersection) / len(df[condition2]), 2) #注意condition2是索引不是df，默认长度等于df的
#         all_pct = round(len(df[condition2]) / len(df), 2) #这里也是
#         hs_xj = -np.log2(sur_pct) * sur_pct - np.log2(1 - sur_pct) * (1 - sur_pct)
#         hs_part = hs_xj * all_pct
#         HS_x.append(hs_part)
#         sur_pc.append([sur_pct, all_pct, hs_xj, ind, condition2, len(df)])
#     uniform = round(1 / len(typ), 2)
#     HS_h0 = -np.log2(uniform) #后面*uniform*num等于乘以1了就不写了
#     HS_observe = sum(HS_x)
#     return HS_h0 - HS_observe

In [7]:
# check_HS_gain('Pclass', newdf)
newdf['Sex'] = newdf['Sex'].astype(float)
newdf.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex            float64
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [8]:
numeric_idx = newdf.dtypes[newdf.dtypes != 'object'].index
label_set = torch.tensor(newdf[numeric_idx]['Survived'], dtype=torch.float32)
newdf_d = newdf[numeric_idx].drop(['Parch', 'PassengerId', 'Survived'], axis=1)
newdf_d = newdf_d.apply(lambda x: (x-x.mean())/x.std())
# newdf_d.plot(kind='scatter', x='Survived', y='Fare')
# plt.show()
newdf_d1 = pd.get_dummies(newdf_d, columns=['Pclass', 'SibSp', 'Sex']).astype(float)
# newdf_d1
dataset = newdf_d1.values
# dataset.shape
newdf_d1

,Age,Fare,Pclass_-1.5652278312782746,Pclass_-0.3691575073769516,Pclass_0.8269128165243713,SibSp_-0.4742788222764234,SibSp_0.4325504280418025,SibSp_1.3393796783600282,SibSp_2.246208928678254,SibSp_3.15303817899648,SibSp_4.059867429314706,SibSp_6.7803551802693836,Sex_-0.7372810452296834,Sex_1.354812621329705
0,-0.555738,-0.502163,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.595147,0.786404,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,-0.268017,-0.488580,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.379356,0.420494,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.379356,-0.486064,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,-0.196086,-0.386454,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
887,-0.771528,-0.044356,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
888,-0.519772,-0.176164,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
889,-0.268017,-0.044356,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [9]:
class Data_test1(Dataset.Dataset):
    def __init__(self, data, label):
        self.data = data
        self.label = label

    def __getitem__(self, index):
        return torch.Tensor(self.data[index]), torch.Tensor(self.label[index])

    def __len__(self):
        return len(self.data)

In [10]:
pct = 0.6
num_train = round(len(dataset) * 0.6)
train_set, train_label = dataset[:num_train], label_set[:num_train]
test_set, test_label = dataset[num_train:], label_set[num_train:]

In [11]:
train_data = Data_test1(train_set, train_label)
test_data = Data_test1(test_set, test_label)

In [12]:
train_iter = DataLoader.DataLoader(dataset=train_data, batch_size=100, shuffle=True)
test_iter = DataLoader.DataLoader(dataset=test_data, batch_size=100, shuffle=True)

In [13]:
loss = nn.CrossEntropyLoss()
class test_net(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential()
        self.net.add_module('linear1', nn.Linear(14, 28))
        self.net.add_module('relu1', nn.ReLU())
        self.net.add_module('linear2', nn.Linear(28, 10))
        self.net.add_module('relu2', nn.ReLU())
        self.net.add_module('dense', nn.Linear(10, 2))
    def forward(self, X):
        for net in self.net:
            X = net(X)
        return X

In [15]:
def train(net, num_epochs, train_iter, test_iter, loss, lr):
    if isinstance(net, nn.Module):
        net.train()
    train_ls = []
    metrics = Accumulator(2)
    optimizer = torch.optim.Adam(net.parameters(), lr=lr)
    for epoch in range(num_epochs):
        for X, y in train_iter:
            y = y.type(torch.long)
            # y = y.reshape((-1, 1))
            # print(X.shape, y.shape)
            optimizer.zero_grad()
            l = loss(net(X), y)
            l.backward()
            optimizer.step()
        metrics.add(float(l * y.numel()), y.numel())
        if epoch % 50 == 0:
            train_ls.append([metrics[0] / metrics[1] , evaluate_accuracy(net, test_iter)]) 
    return train_ls

In [16]:
def accuracy(y_hat, y):
    if len(y_hat.shape) > 1 and y_hat.shape[1] > 1:
        y_hat = y_hat.argmax(axis=1)
    cmp = y_hat.type(y.dtype) == y
    ##自制auc曲线看一下 TPP和FPP不是除以所有！是除以True P!   True F!
    auc = []
    counttp = 0
    countfp = 0
    for i in range(y.numel()):
        if y[i] - y_hat.type(y.dtype)[i] == -1:
            countfp += 1
        if (y[i] - y_hat.type(y.dtype)[i] == 0) and (y[i] == 1):
            counttp += 1
    auc.append([counttp / y[y==1].sum(), countfp / y[y==0].sum()])
    return float(cmp.type(y_hat.dtype).sum()), auc

In [17]:
class Accumulator:
    def __init__(self, n):
        self.data = [0.0] * n

    def add(self, *args):
        self.data = [a + float(b) for a, b in zip(self.data, args)]

    def reset(self):
        self.data = [0.0] * len(self.data)

    def __getitem__(self, index):
        return self.data[index]


In [18]:
def evaluate_accuracy(net, data_iter):
    if isinstance(net, nn.Module):
        net.eval()
    metric = Accumulator(2)
    for X, y in data_iter:
        metric.add(accuracy(net(X), y)[0], y.numel())
    return metric[0] / metric[1], accuracy(net(X), y)[1]

In [19]:
# for i, (data, label) in enumerate(train_iter):
#     if i==2:
#         break
#     print(net(data), label.reshape((-1, 1)))

In [22]:
net = test_net()
# test_X = torch.ones((2, 12))
# net.parameters()
train(net, 500, train_iter, test_iter, loss, 0.02)
# net

[[0.6010979788643973, (0.7780898876404494, [[tensor(0.5263), tensor(inf)]])],
 [0.4163650881342527, (0.797752808988764, [[tensor(0.6667), tensor(inf)]])],
 [0.3791734845162787, (0.8033707865168539, [[tensor(0.5652), tensor(inf)]])],
 [0.3519399956926041, (0.800561797752809, [[tensor(0.6667), tensor(inf)]])],
 [0.345705811487671, (0.797752808988764, [[tensor(0.7692), tensor(inf)]])],
 [0.33626630861242185, (0.7808988764044944, [[tensor(0.6250), tensor(inf)]])],
 [0.33048127513391234, (0.7865168539325843, [[tensor(0.6087), tensor(inf)]])],
 [0.3193170993464916, (0.797752808988764, [[tensor(0.7200), tensor(inf)]])],
 [0.31560911692288746, (0.7837078651685393, [[tensor(0.7273), tensor(inf)]])],
 [0.3105713377279838, (0.7893258426966292, [[tensor(0.8095), tensor(inf)]])]]

In [35]:
a = torch.empty(3, dtype=torch.long).random_(3)
b = torch.randn((3, 3)).argmax(axis=1)
a, b, b.dtype

(tensor([1, 0, 0]), tensor([0, 1, 0]), torch.int64)

In [32]:
a = torch.tensor([[1, 0], [0, 1], [1, 0]])
b = torch.tensor([1, 0, 1])
a[a.argmax(axis=1)==0].sum()

tensor(2)